# Binary Classification Realization with MLP

In [1]:
###########################
# With One Hidden Layer
###########################

In [2]:
%run ../Regression_Analysis/SLP_REGRESSION.ipynb

Epoch 1 : loss = 35.182, accuracy = 0.547/0.803
Epoch 2 : loss = 8.874, accuracy = 0.814/0.809
Epoch 3 : loss = 8.134, accuracy = 0.805/0.803
Epoch 4 : loss = 8.017, accuracy = 0.802/0.804
Epoch 5 : loss = 7.924, accuracy = 0.803/0.802
Epoch 6 : loss = 7.843, accuracy = 0.802/0.803
Epoch 7 : loss = 7.767, accuracy = 0.802/0.804
Epoch 8 : loss = 7.697, accuracy = 0.802/0.806
Epoch 9 : loss = 7.634, accuracy = 0.805/0.804
Epoch 10 : loss = 7.574, accuracy = 0.804/0.805

 Final Test : final accuracy = 0.805
[[0.55842942]
 [0.76572744]
 [0.00429812]
 [2.24857982]
 [1.78899292]
 [0.66286655]
 [2.723841  ]
 [0.65809305]
 [0.54706306]
 [1.1225188 ]]
[4.52425871]
Epoch 1 : loss = 7.521, accuracy = 0.803/0.757
Epoch 2 : loss = 6.212, accuracy = 0.817/0.803
Epoch 3 : loss = 5.859, accuracy = 0.824/0.835
Epoch 4 : loss = 5.598, accuracy = 0.828/0.838
Epoch 5 : loss = 5.440, accuracy = 0.830/0.838
Epoch 6 : loss = 5.240, accuracy = 0.834/0.796
Epoch 7 : loss = 5.201, accuracy = 0.834/0.850
Epoch 8

In [3]:
# Init model (hidden layer 1)
def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
    
    # parameter for hidden layer(weight and bias)
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt])
    
    # parameter for output layer(weight and bias)
    pm_output = alloc_param_pair([hidden_cnt, output_cnt])

def alloc_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias}

"""
each layer of the neural network needs a pair of parameters consisting of a weight
matrix and a bias vector, as in a single layer perceptron.
alloc_param_pair fucntion creates and initializes a parameter pair for one layer.
"""

'\neach layer of the neural network needs a pair of parameters consisting of a weight\nmatrix and a bias vector, as in a single layer perceptron.\nalloc_param_pair fucntion creates and initializes a parameter pair for one layer.\n'

In [4]:
def forward_neuralnet_hidden1(x):
    global pm_output, pm_hidden
    
    # hidden vector
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    
    # output vector
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
        
    return output, [x, hidden]

def relu(x):
    return np.maximum(x, 0)

"""
forward neural net hidden1 function's hidden vector return value is used for 
calculating partial differentiation in backward propagation process
"""

"\nforward neural net hidden1 function's hidden vector return value is used for \ncalculating partial differentiation in backward propagation process\n"

In [31]:
def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    
    # aux is forward_neuralnet_hidden1 function's second return
    # aux = [x, hidden] ; x and hidden vector
    x, hidden = aux
    
    ################################################################################
    # backprop 1
    ################################################################################
    
    # G_output = delta L / delta Y
    g_output_w_out = hidden.transpose()
    
    # G_w_out = delta L / delta W = (delta Y / delta W) * (delta L / delta Y)
    # = g_output_w_out * G_output
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis = 0)
    
    g_output_hidden = pm_output['w'].transpose()
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out  
    
    ################################################################################
    # backprop 2
    ################################################################################
    
    G_hidden = G_hidden * relu_derv(hidden)
    
    g_hidden_w_hid = x.transpose()
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
    G_b_hid = np.sum(G_hidden, axis = 0)
    
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid

In [6]:
def relu_derv(y):
    return np.sign(y)

"""
when y has positive value -> np.sign(y) return 1
when y has zero valeu -> np.sign(y) return 0

y has only two types, zero or positive
"""

'\nwhen y has positive value -> np.sign(y) return 1\nwhen y has zero valeu -> np.sign(y) return 0\n\ny has only two types, zero or positive\n'

In [7]:
###########################
# With Multi Hidden Layer
###########################

In [8]:
def init_model_hiddens():
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    
    pm_hiddens = []
    prev_cnt = input_cnt
    
    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
    
    pm_output = alloc_param_pair([prev_cnt, output_cnt])

In [9]:
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    
    # hidden vectors
    hidden = x
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['x']) + pm_hidden['b'])
        hiddens.append(hidden)
    
    # output vector
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens

In [10]:
def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens
    
    # aux is forward_neuralnet_hidden1 function's second return
    # aux = [output, hiddens]
    # hiddens = hidden vectors [x(input), hidden1(1st hidden layer vector), ...]
    hiddens = aux
    
    ################################################################################
    # backprop for ouptut - hidden layers
    ################################################################################
    
    # G_output = delta L / delta Y
    g_output_w_out = hiddens[-1].transpose()
    
    # G_w_out = delta L / delta W = (delta Y / delta W) * (delta L / delta Y)
    # = g_output_w_out * G_output
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis = 0)
    
    g_output_hidden = pm_output['w'].transpose()
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    ################################################################################
    # backprop for hidden layers - input
    ################################################################################
    
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n + 1])
        
        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis = 0)
        
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)
        
        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid

In [11]:
global hidden_config

def init_model():
    if hidden_config is not None:
        print('MLP with {} hidden layers is executed.' .format(len(hidden_config)))
        
        init_model_hiddens()
    else:
        print('MLP with 1 hidden layer is executed')
        
        init_model_hidden1()

def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden1(x)
    
def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        return backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        return backprop_neuralnet_hidden1(G_output, hiddens)

In [12]:
def set_hidden(info):
    global hidden_cnt, hidden_config
    
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None
    else:
        hidden_config = info


In [13]:
def eval_accuracy(output, y):
   
    mdiff = np.mean(np.abs((output - y)/y))
    
    return 1 - mdiff

In [14]:
def backprop_postproc(G_loss, diff):
    
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1
    
    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output  

In [34]:
set_hidden([])
abalone_exec()

MLP with 0 hidden layers is executed.
Epoch 1 : loss = 7.391, accuracy = 0.807/0.775
Epoch 2 : loss = 6.065, accuracy = 0.821/0.811
Epoch 3 : loss = 5.852, accuracy = 0.822/0.832
Epoch 4 : loss = 5.517, accuracy = 0.829/0.844
Epoch 5 : loss = 5.264, accuracy = 0.833/0.832
Epoch 6 : loss = 5.252, accuracy = 0.834/0.848
Epoch 7 : loss = 5.217, accuracy = 0.834/0.815
Epoch 8 : loss = 5.108, accuracy = 0.836/0.812
Epoch 9 : loss = 5.095, accuracy = 0.836/0.843
Epoch 10 : loss = 5.054, accuracy = 0.836/0.839

 Final Test : final accuracy = 0.839


In [35]:
set_hidden(4)
abalone_exec(epoch_count = 50, report = 10)

MLP with 1 hidden layer is executed
Epoch 10 : loss = 10.545, accuracy = 0.728/0.734
Epoch 20 : loss = 10.533, accuracy = 0.729/0.719
Epoch 30 : loss = 10.561, accuracy = 0.728/0.683
Epoch 40 : loss = 10.550, accuracy = 0.728/0.748
Epoch 50 : loss = 10.547, accuracy = 0.727/0.753

 Final Test : final accuracy = 0.753


In [36]:
abalone_exec(epoch_count=50, report = 10)

MLP with 1 hidden layer is executed
Epoch 10 : loss = 10.523, accuracy = 0.729/0.713
Epoch 20 : loss = 10.556, accuracy = 0.729/0.695
Epoch 30 : loss = 10.518, accuracy = 0.729/0.712
Epoch 40 : loss = 10.468, accuracy = 0.730/0.743
Epoch 50 : loss = 10.538, accuracy = 0.728/0.718

 Final Test : final accuracy = 0.718
